In [ ]:
!pip install pymupdf pytesseract pdf2image opencv-python pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 19.1 MB/s eta 0:00:00


In [ ]:
!apt-get install -y poppler-utils
!apt-get install -y tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (224 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123623 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
T

In [ ]:
import pytesseract
import os
import glob
import logging
from datetime import datetime
from PIL import Image
import pytesseract
import pymupdf as fitz
from pdf2image import convert_from_path
import numpy as np
import cv2
import io
import os
import shutil
from PIL import Image

In [ ]:
now = datetime.now()
dt_string = now.strftime("%d_%m_%Y %H_%M_%S")

logging.basicConfig(filename='/content/logs/log_' + dt_string + '.conf', filemode='a',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S', level=logging.INFO)

logger = logging.getLogger('OCR')
logging.info("Running OCR")

In [ ]:
output_dir = '/content/extracted_writeup'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
import fitz
def extract_text_from_pdf(pdf_path):
    all_text = ""
    try:
        doc = fitz.open(pdf_path)
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            text = page.get_text()
            all_text += text + "\n"

            # If there's no text, try to extract text from images using OCR
            if len(text.strip()) == 0:
                image_list = page.get_images(full=True)
                for _, img in enumerate(image_list):
                    base_image = doc.extract_image(img[0])
                    img = Image.open(io.BytesIO(base_image["image"]))
                    text_from_image = pytesseract.image_to_string(img)
                    all_text += text_from_image + "\n"
    except Exception as e:
        logger.error(f"Error processing PDF {pdf_path}: {e}", exc_info=True)
    return all_text


In [ ]:
from google.colab import files

def process_uploaded_files(uploaded_files):
    for file_name in uploaded_files.keys():
        logger.info(f"Processing file: {file_name}")

        # Extract text from PDF
        if file_name.lower().endswith('.pdf'):
            extracted_text = extract_text_from_pdf(file_name)
            output_file_path = os.path.join(output_dir, file_name.split(".")[0] + ".txt")
            with open(output_file_path, 'w') as write_file:
                write_file.write(extracted_text)
            logger.info(f"Extracted text saved to: {output_file_path}")

        elif file_name.lower().endswith('.png'):
            try:
                img = Image.open(file_name)
                text_from_image = pytesseract.image_to_string(img)
                output_file_path = os.path.join(output_dir, file_name.split(".")[0] + ".txt")
                with open(output_file_path, 'w') as write_file:
                    write_file.write(text_from_image)
                logger.info(f"Text extracted from image and saved to: {output_file_path}")
            except Exception as e:
                logger.error(f"Error processing image {file_name}: {e}", exc_info=True)
        else:
            logger.warning(f"Unsupported file type for OCR: {file_name}")


In [ ]:
uploaded = files.upload()

# Log the uploaded files
uploaded_files = list(uploaded.keys())
logger.info(f"Uploaded files: {uploaded_files}")

# Process the uploaded files
process_uploaded_files(uploaded)

# Optionally, print the files in the extracted writeup folder to confirm saving
print(f"Extracted files saved to: {output_dir}")
print(os.listdir(output_dir))

Saving Adhaar card.pdf to Adhaar card.pdf
Extracted files saved to: /content/extracted_writeup
['Adhaar card.txt']


In [ ]:
for pdf_file in uploaded.keys():
    print("**************" + pdf_file + "**************")
    extracted_text = extract_text_from_pdf(pdf_file)
    print(extracted_text)

    with open(f'/content/extracted_writeup{pdf_file.split(".")[0]}.txt', 'w') as write_file:
        write_file.write(extracted_text)


**************Adhaar card - shubham jankar.png**************




In [ ]:
# import shutil

# # Create directory for output files
# os.makedirs('/content/extracted_writeup1', exist_ok=True)

# # Move files to a specific folder (for easy download)
# shutil.move('/content/extracted_writeup1', '/content/extracted_writeup.zip')

# # Zip the folder
# shutil.make_archive('/content/extracted_writeup1', 'zip', '/content', 'extracted_writeup1')

# # Provide download link
# files.download('/content/extracted_writeup.zip')
